In [1]:
import glob
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import cartopy
import numpy as np
import cartopy.crs as ccrs
from pyproj import Proj, Transformer
from scipy.spatial.kdtree import KDTree
import datetime
import os
import xarray as xr
import tqdm
import warnings
warnings.filterwarnings("ignore")

/var/folders/_1/fh2vf5m57871c546k_sf7xr80000gn/T/ipykernel_14978/3502479102.py:8: DeprecationWarning: Please use `KDTree` from the `scipy.spatial` namespace, the `scipy.spatial.kdtree` namespace is deprecated.
  from scipy.spatial.kdtree import KDTree


In [2]:
def lonlat_to_xy(coords_1, coords_2, hemisphere, inverse=False):

    """Converts between longitude/latitude and EASE xy coordinates.
 
    Args:
        lon (float): WGS84 longitude
        lat (float): WGS84 latitude
        hemisphere (string): 'n' or 's'
        inverse (bool): if true, converts xy to lon/lat
 
    Returns:
        tuple: pair of xy or lon/lat values
    """

    EASE_Proj = {'n': 'EPSG:3408',
                 's': 'EPGS:3409'}
    
    WGS_Proj = 'EPSG:4326'
    
    for coords in [coords_1, coords_2]: assert isinstance(coords,(np.ndarray,list))

    if inverse == False: # lonlat to xy
        
        lon, lat = coords_1, coords_2
        
        transformer = Transformer.from_crs(WGS_Proj, EASE_Proj[hemisphere])
        
        x, y = transformer.transform(lat, lon)
        
        return (x, y)

    else: # xy to lonlat
        
        x, y = coords_1, coords_2
        
        transformer = Transformer.from_crs(EASE_Proj[hemisphere], WGS_Proj)
        
        lat, lon = transformer.transform(x, y)
        
        return (lon, lat)

In [3]:
# load ease lons,lats:
ease_lats = np.load('/Users/carmennab/Dropbox/alpha_retracker/data/auxiliary/lat_25km_cent.npy')
ease_lons = np.load('/Users/carmennab/Dropbox/alpha_retracker/data/auxiliary/lon_25km_cent.npy')

# convert into x,y:
ease_x, ease_y = lonlat_to_xy(ease_lons,ease_lats,hemisphere='n')

# create KDTree to find EASE grid cell closest to buoy:
tree = KDTree(list(zip(ease_x.ravel(),ease_y.ravel())))

In [ ]:
datapath = '/Users/carmennab/cpom_server/home/cjn/pySnowRadar/Arctic/OIB_processed/'

days = np.arange(0,4018)
dates = [datetime.date(2010,1,1)+datetime.timedelta(days=np.int(days)) for days in days]

for day in days:
    oib_snow = np.full((ease_x.shape),np.nan)
    inds_sn = {}
    date = dates[day].strftime('%Y%m%d')
 
    for f in os.listdir(datapath):
        if date in f:

            data = Dataset(datapath+f)

            snow = np.array(data['snow_depth'])
            oib_lons = np.array(data['lon'])
            oib_lats = np.array(data['lat'])
            snow[snow>1.5] = np.nan

            oib_x,oib_y = lonlat_to_xy(oib_lons,oib_lats,hemisphere='n') 

            for x,y,sn in zip(oib_x,oib_y,snow):

                dist, ind = tree.query([x,y])

                ind2d = np.unravel_index(ind, (ease_lats.shape[0],ease_lats.shape[1]))

                if ind2d in inds_sn.keys():
                    inds_sn[ind2d].append(sn)
                else:   
                    inds_sn[ind2d] = [sn]

    for key in inds_sn.keys():
        oib_snow[key[0],key[1]]=np.nanmean(inds_sn[key])

    if (oib_snow[~np.isnan(oib_snow)].shape[0]>0):
        
        filepath = '/Users/carmennab/Dropbox/alpha_retracker/data/pySnowRadar_EASE/'+date+'.nc'
        ! rm {filepath} 
        data_vars =  {'Snow Depth':(['x','y'],oib_snow)}
        ds = xr.Dataset(data_vars = data_vars,
                        coords={'Longitude':(['x','y'],ease_lons),
                                'Latitude':(['x','y'],ease_lats)},
                        )
        ds.to_netcdf(filepath,'w')